In [2]:
# setting device on GPU if available, else CPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

# Additional Info when using cuda
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory Usage:")
    print("Allocated:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), "GB")
    print("Cached:   ", round(torch.cuda.memory_reserved(0) / 1024**3, 1), "GB")

Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from functools import partial
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

## test rendering

In [5]:
def findAllFile(base):
    """
    Recursively find all files in the specified directory.

    Args:
        base (str): The base directory to start the search.

    Returns:
        list: A list of file paths found in the directory and its subdirectories.
    """
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path

In [6]:
from utils.motion_processing.hml_process import recover_from_ric, recover_root_rot_pos
import utils.vis_utils.plot_3d_global as plot_3d
import matplotlib.pyplot as plt

In [7]:
from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser
from core.models.generation.lm import LMModel, MotionGen

from configs.config import cfg, get_cfg_defaults
from configs.config_streaming import get_cfg_defaults as strm_get_cfg_defaults


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
gen_cfg = strm_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_streaming/motion_streaming.yaml")
gen_cfg.freeze()

In [9]:

body_cfg = get_cfg_defaults()
body_cfg.merge_from_file(gen_cfg.vqvae.body_config)
left_cfg = get_cfg_defaults()
left_cfg.merge_from_file(gen_cfg.vqvae.left_hand_config)
right_cfg = get_cfg_defaults()
right_cfg.merge_from_file(gen_cfg.vqvae.right_hand_config)

In [10]:
from core.models.resnetVQ.vqvae import HumanVQVAE


In [11]:
left_hand_model = HumanVQVAE(left_cfg.vqvae).to(device).eval()
left_hand_model.load(os.path.join(left_cfg.output_dir, "vqvae_motion.pt"))

right_hand_model = HumanVQVAE(right_cfg.vqvae).to(device).eval()
right_hand_model.load(os.path.join(right_cfg.output_dir, "vqvae_motion.pt"))

body_model = HumanVQVAE(body_cfg.vqvae).to(device).eval()
body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))



### VQVAE

In [12]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset as load_dataset_vq
from core.datasets.vq_dataset import simple_collate as simple_collate_vq


In [13]:
def get_decoded(model , motion):
    encs = []
    inds = []
    for i in range(0 , motion.shape[1] , 120 ):
        if i + 240 >= motion.shape[1]:
            enc_b = model(motion[: , i:, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
            break
        else:
            enc_b = model(motion[: , i:i + 120, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
    return torch.cat(encs , 1), torch.cat(inds , -1)

In [14]:
condition_provider2 = ConditionProvider(
            motion_rep=MotionRep("full"),
            motion_padding="max_length",

        )

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [15]:
body_cfg.dataset.window_size = -1

In [16]:
test_ds, _, _ = load_dataset_vq(
            # dataset_names=["choreomaster"],
            dataset_args=body_cfg.dataset,
            split="train",
        )

Total number of motions animation: 312
Total number of motions humanml: 19984
Total number of motions perform: 451
Total number of motions GRAB: 1268
Total number of motions idea400: 11886
Total number of motions humman: 706
Total number of motions beat: 1458
Total number of motions game_motion: 9699
Total number of motions music: 3385
Total number of motions aist: 1396
Total number of motions fitness: 15886
Total number of motions moyo: 161
Total number of motions choreomaster: 34
Total number of motions dance: 153
Total number of motions kungfu: 987
Total number of motions EgoBody: 931
Total number of motions HAA500: 4969


In [17]:
dl = torch.utils.data.DataLoader(
            test_ds,
            batch_size=1,
            sampler=None,
            shuffle=False,
            collate_fn=partial(simple_collate_vq , conditioner = condition_provider2),
            
        )

In [ ]:
errors = []
for batch in tqdm(dl):
    sve = os.path.join("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body_rv" , batch["names"][0] )
    if os.path.exists(sve):
        continue
    os.makedirs(os.path.dirname(sve) , exist_ok=True)
    try:
        enc_b, inb = get_decoded(body_model , batch["motion"][0].to(device))
        np.save(sve , inb.cpu().numpy())
    except:
        errors.append(batch["names"][0])
    

 69%|████████████████████████████████████████████████████████▏                         | 50464/73666 [12:08<28:08, 13.74it/s]

In [20]:
len(errors)

30993